In [15]:
import cv2
import numpy as np
from numpy import shape
import matplotlib.pyplot as plt 
from image_extractor import select_feature, feature_extract

# image_dir  = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Agglutinated/image (2).JPG'
# image_dir  = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Brittle/image (87).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Plated_Purple/image (10).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Compartmentalized_Purple/image (100).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Compartmentalized_White/image (24).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Plated_White/image (72).JPG'
image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Plated_Slaty/image (71).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Compartmentalized_Slaty/image (56).JPG'
# image_dir = 'D:/Thesis_data/Backups/Color_Corrected_512x512/Moldered/image (7).JPG'

plt.rcParams['figure.dpi'] = 102.46 

In [16]:
# features = select_feature(image_dir)
# print(type(features))

In [17]:
extractor = feature_extract()
image_bgr = cv2.imread(image_dir)
extractor.process(image_bgr)


In [22]:
print((extractor.structure))

[ 8.30300425e+01  1.68025805e+02  1.22120454e+01  1.01232140e+01
  1.38620605e+02  2.20582367e+02  2.69516212e+00  1.92460666e+01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  9.05000000e+01  2.46000000e+02  4.00000000e+00  2.10000000e+01
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3.08888126e+01  7.01054722e+01  1.12350570e+01  9.87211292e+00
  1.17942810e+00  7.11823425e+01  9.06307876e-01  4.93523264e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  1.87788819e-01  3.92972509e-01  1.26421494e+00  1.24605615e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000

In [19]:
print(extractor.mold)

[ 7.84210526e+00  1.05000000e+01  1.09360671e+02  1.05273005e+02
  6.69593585e+01  5.12637427e+01  4.87051859e+01  3.20886448e+01
 -1.18060602e-01 -8.08597819e-02  5.34688174e-02 -1.09203143e+00
 -1.08633609e+00 -8.68412757e-01  3.61667422e+00  7.52102317e+00
  1.67848359e+01  1.28170756e+01  1.29126456e+01  2.47702812e+01
  2.27888540e+01  8.49178628e+00  5.47908584e-01  5.27484025e-01
  4.53520112e-01  4.66359201e-01  4.71645596e-01  4.54887182e-01
  1.40937162e+00  2.11586481e+00  1.58104388e+00  3.62584069e-01
 -2.13737323e-01  4.38664130e-02  8.37776728e-02 -5.32802542e-02
  7.27772915e-02  3.23708310e-02 -1.70237914e-01  8.76682737e-01
  8.36509200e-01  7.32670029e-01  1.07649425e+00  3.86356586e+00
  5.90927303e+00  3.83865460e+00 -1.17296398e+00 -1.87617099e+00
 -1.81857498e+00 -9.70797830e-01 -8.90531694e-01 -7.59251810e-01
 -3.46081908e-01  1.80064263e-01 -1.14073799e-02 -2.68576021e-01]


In [20]:
print(extractor.overall_geometry)

[171, 82, 171, 349, 45677.0, 970.6072062253952, 170.9042205810547, 342.262451171875, 1.7893288378110643, 47249.0, 174.2356719970703, 0.9667294545916316, 0.7653781062015114, 1.839204506189209]


In [21]:
print(extractor.overall_statistic)

[67.18516    66.938416   46.0716     46.471848   44.016083   29.156612
  0.22330204 -0.12166926 -0.4317345   0.1067353  -0.30239233 -0.5968161 ]
